<a href="https://cognitiveclass.ai"><img src = "http://seekvectorlogo.com/wp-content/uploads/2019/01/coursera-vector-logo.png" width = 200> </a>

<a href="https://cognitiveclass.ai"><img src = "http://www.logospng.com/images/80/ibmwatsonlogo-cogx-london-2018-80659.png" width = 200> </a>


# Segmenting and Clustering Neighborhoods in Toronto

#### Coursera Capstone Project, Week 3

## Introduction

In this lab, I learned how to convert addresses into their equivalent latitude and longitude values. Also, I used BeautifulSoup to web scrape a Wikipedia Table to obtain the postal codes for Toronto. I used the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. I used the *k*-means clustering algorithm to complete this task. Finally, I used the Folium library to visualize the neighborhoods in Toronto and their emerging clusters.

## 1. Download and Explore Dataset

### Load Libraries and Obtain data from Wikipedia Table

Scraping Wikipedia Table is performed with BeautifulSoup and Request libraries [1]

In [1]:
import numpy as np # library to handle data in a vectorized manner

from bs4 import BeautifulSoup # Library to web scrapping wikipedia tables
import requests # Library to get http requests

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

#pd.options.display.max_columns = 20

Libraries imported.


In [2]:
### Web Scrapping Wikipedia Table

WIKI_PAGE = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Grab the page HTML and get the list of rows for the table
req = requests.get(WIKI_PAGE)
page_html = BeautifulSoup(req.text, 'lxml')
wiki_table = page_html.find('table', attrs = {'class':'wikitable sortable'})
row_list = wiki_table.find_all('tr')

# First row in the table is the header, so extract that separately
header_row = row_list.pop(0)
header_th = header_row.find_all('th')
header = [el.text for el in header_th]

table_dict = {x:[] for x in header}

# Now for the rest of the table...
for row in row_list:
    row_td = row.find_all('td')
    for el,td in zip(header,row_td):
        table_dict[el].append(td.text)

df = pd.DataFrame(table_dict)

### Cleaning the resulting Dataframe

Resulting Dataframe includes '\n' values at the last column, we need to replace them to extract the correct information.

In [3]:
## Clean dataframe
df['Neighbourhood\n'] = df['Neighbourhood\n'].str.replace('\n','')

## Replaces \n's
df.columns = df.columns.str.replace('\n','')

print (df.head())

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront


In [4]:
## Filter Not assigned Boroughs
df2 = df[df['Borough'] != 'Not assigned']

In [5]:
## Group Neighbourhoods with same Postal Code
df3 = df2.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

def CheckNeihgborhood(x):
    if x['Neighbourhood'] == 'Not assigned':
        x['Neighbourhood'] = x['Borough']
    else:
        x['Neighbourhood']
    return x['Neighbourhood']

## Assign Neighbourhood = Borough if Neighbourhood have Not assigned labels
df3['Neighbourhood'] = df3.apply(CheckNeihgborhood, axis=1)

In [6]:
print (df3.head())
print (df3.shape)

  Postcode      Borough                           Neighbourhood
0      M1B  Scarborough                          Rouge, Malvern
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2      M1E  Scarborough       Guildwood, Morningside, West Hill
3      M1G  Scarborough                                  Woburn
4      M1H  Scarborough                               Cedarbrae
(103, 3)


### Obtaining Lattitude and Longitude with Geocoder library

I figured out that the Geocoder library was not getting information with the default geocoder.google() function, insetad of that I found that the 'arcgis' data source was working better and obtained results.

Also the proposed function to obtain lattitude and longitude for the cities was adapted as a defined function to be applied among the resulting dataframe.

In [7]:
### Add lattitde and longitude
import geocoder # import geocoder

def GetCoord(Borough,Postcode):
    g = geocoder.arcgis('{}, ON {}'.format(Borough,Postcode))
    lat_long_coords = g.latlng
    return lat_long_coords

df3['LatLong'] = df3.apply(lambda x: GetCoord(x['Borough'],x['Postcode']), axis=1)
df3[['Lattitude','Longitude']] = pd.DataFrame(df3['LatLong'].values.tolist())
df3 = df3.drop('LatLong', 1)

df3.head(10)

,Postcode,Borough,Neighbourhood,Lattitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696665,-79.260163


In [8]:
print (df3.columns.tolist(),'\n')
df3.info()

['Postcode', 'Borough', 'Neighbourhood', 'Lattitude', 'Longitude'] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
Postcode         103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
Lattitude        103 non-null float64
Longitude        103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


(Web scrapping process and Coordinates obtaining successfully complete!)

### Map visualization with Folium

#### Create a map of Toronto with neighborhoods superimposed on top

Use geopy library to get the latitude and longitude values of Toronto.

In [9]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [10]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Lattitude'], df3['Longitude'], df['Borough'], df3['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [11]:
CLIENT_ID = 'ZURNZQ4NR1HSOQOYFF5BMRCQC2N4K5VV0AGPNU5K1QA4UAHB' # your Foursquare ID
CLIENT_SECRET = 'TLA32TBUQLKQL2HFV0TA4JYRKFKKWMY5RVQUVVIH4E1DQNUI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: ZURNZQ4NR1HSOQOYFF5BMRCQC2N4K5VV0AGPNU5K1QA4UAHB
CLIENT_SECRET:TLA32TBUQLKQL2HFV0TA4JYRKFKKWMY5RVQUVVIH4E1DQNUI


#### Let's explore the first neighborhood in our dataframe.

In [12]:
df3.loc[0, 'Neighbourhood']

'Rouge, Malvern'

In [13]:
neighborhood_latitude = df3.loc[0, 'Lattitude'] # neighborhood latitude value
neighborhood_longitude = df3.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df3.loc[0, 'Neighbourhood'] # neighborhood name

print('Lattitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Lattitude and longitude values of Rouge, Malvern are 43.811525000000074, -79.19551721399995.


#### Now, let's get the top 100 venues that are in Rouge, Malvern within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [14]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 800 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZURNZQ4NR1HSOQOYFF5BMRCQC2N4K5VV0AGPNU5K1QA4UAHB&client_secret=TLA32TBUQLKQL2HFV0TA4JYRKFKKWMY5RVQUVVIH4E1DQNUI&v=20180605&ll=43.811525000000074,-79.19551721399995&radius=800&limit=100'

Send the GEt request and examine the results,

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ca8f6a7f594df2400f19dfa'},
 'response': {'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.81872500720008,
    'lng': -79.1855582964134},
   'sw': {'lat': 43.80432499280007, 'lng': -79.2054761315865}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c97975582b56dcb8320ebaa',
       'name': 'Canadiana exhibit',
       'location': {'lat': 43.81796218928876,
        'lng': -79.19337359666939,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.81796218928876,
          'lng': -79.19337359666939}],
        'distance': 736,
        'cc': 'CA',
        'city': 'Toronto',
        'state': 'ON',
 

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [17]:
import json # library to handle JSON files

venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.io.json.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Canadiana exhibit,Zoo Exhibit,43.817962,-79.193374
1,Wendy's,Fast Food Restaurant,43.807448,-79.199056
2,Grizzly Bear Exhibit,Zoo Exhibit,43.817031,-79.193458
3,Grizzly Bear Trail,Zoo Exhibit,43.818034,-79.192108


In [18]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [20]:
toronto_venues = getNearbyVenues(names=df3['Neighbourhood'],
                                   latitudes=df3['Lattitude'],
                                   longitudes=df3['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [21]:
print(toronto_venues.shape)
toronto_venues.head(10)

(2878, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750,Scarborough Historical Society,43.788755,-79.162438,History Museum
1,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.765690,-79.175256,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.765690,-79.175256,Heron Park Community Centre,43.768867,-79.176958,Gym / Fitness Center
4,"Guildwood, Morningside, West Hill",43.765690,-79.175256,Heron Park,43.769327,-79.177201,Park
5,Woburn,43.768359,-79.217590,Starbucks,43.770037,-79.221156,Coffee Shop
6,Woburn,43.768359,-79.217590,cheapOseo,43.766042,-79.218539,Business Service
7,Woburn,43.768359,-79.217590,Korean Grill House,43.770812,-79.214502,Korean Restaurant
8,Woburn,43.768359,-79.217590,Densgrove Park,43.765397,-79.220130,Park
9,Cedarbrae,43.769688,-79.239440,North Bendale Park,43.769963,-79.245265,Playground


In [22]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,12,12,12,12,12,12
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",4,4,4,4,4,4
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",13,13,13,13,13,13
"Alderwood, Long Branch",4,4,4,4,4,4
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",20,20,20,20,20,20
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",6,6,6,6,6,6


#### Let's find out how many unique categories can be curated from all the returned venues

In [23]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 204 uniques categories.


## 3. Analyze Each Neighborhood

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(10)

,Yoga Studio,Accessories Store,Airport,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Store,Bike Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Great Outdoors,Park,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [25]:
toronto_onehot.shape

(2878, 204)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Store,Bike Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Carpet Store,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Great Outdoors,Park,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.01,0.0,0.030000,0.000000,0.000000,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.020000,0.000000,0.010000,0.020000,0.000000,0.000000,0.000000,0.01,0.030000,0.010000,0.000000,0.0,0.00,0.00,0.000000,0.060000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.050000,0.000000,0.000000,0.000000,0.01,0.000000,0.020000,0.000000,0.000000,0.020000,0.000000,0.010000,0.00,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.01,0.000000,0.01,0.000000,0.000000,0.010000,0.000000,0.020000,0.010000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.040000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010

#### Let's confirm the new size

In [27]:
toronto_grouped.shape

(100, 204)

#### Let's print each neighborhood along with the top 5 most common venues

In [28]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
          venue  freq
0          Café  0.06
1   Coffee Shop  0.05
2         Hotel  0.04
3  Burger Joint  0.03
4        Bakery  0.03


----Agincourt----
                venue  freq
0       Shopping Mall  0.17
1         Supermarket  0.17
2                Pool  0.08
3  Chinese Restaurant  0.08
4              Bakery  0.08


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0            Pharmacy  0.75
1    Sushi Restaurant  0.25
2         Yoga Studio  0.00
3           Nightclub  0.00
4  Mexican Restaurant  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store  0.15
1                  Park  0.08
2  Fast Food Restaurant  0.08
3            Beer Store  0.08
4        Sandwich Place  0.08


----Alderwood, Long Branch----
            venue  freq
0  Sandwich Place  0.25
1             Pub  0.25
2    Da

                venue  freq
0   Convenience Store  0.25
1  Italian Restaurant  0.25
2      Farmers Market  0.25
3                Park  0.25
4              Office  0.00


----Emery, Humberlea----
          venue  freq
0   Coffee Shop  0.50
1          Park  0.25
2     Nightclub  0.25
3   Yoga Studio  0.00
4  Noodle House  0.00


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.18
1  Fast Food Restaurant  0.10
2           Coffee Shop  0.04
3            Food Court  0.04
4      Toy / Game Store  0.04


----First Canadian Place, Underground city----
          venue  freq
0          Café  0.06
1   Coffee Shop  0.05
2         Hotel  0.04
3  Burger Joint  0.03
4        Bakery  0.03


----Flemingdon Park, Don Mills South----
           venue  freq
0  Grocery Store  0.29
1   Intersection  0.14
2     Smoke Shop  0.14
3    Coffee Shop  0.14
4            Gym  0.14


----Forest Hill North, Forest Hill West----
                       venue  freq
0          

          venue  freq
0          Café  0.06
1   Coffee Shop  0.05
2         Hotel  0.04
3  Burger Joint  0.03
4        Bakery  0.03


----Studio District----
                 venue  freq
0               Bakery  0.06
1   Italian Restaurant  0.06
2                Diner  0.06
3     Sushi Restaurant  0.04
4  American Restaurant  0.04


----The Annex, North Midtown, Yorkville----
                       venue  freq
0                       Park  0.43
1          Convenience Store  0.29
2                       Bank  0.14
3              Grocery Store  0.14
4  Middle Eastern Restaurant  0.00


----The Beaches----
                       venue  freq
0            Thai Restaurant  0.25
1                       Bank  0.25
2                      Diner  0.25
3             Discount Store  0.25
4  Middle Eastern Restaurant  0.00


----The Beaches West, India Bazaar----
            venue  freq
0  Sandwich Place  0.10
1             Pub  0.05
2      Steakhouse  0.05
3   Burrito Place  0.05
4     Coffee Shop  

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [30]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Coffee Shop,Hotel,Burger Joint,Bar,Bakery,Asian Restaurant,Steakhouse,Japanese Restaurant,Restaurant
1,Agincourt,Supermarket,Shopping Mall,Park,Pool,Sushi Restaurant,Chinese Restaurant,Shanghai Restaurant,Bakery,Hong Kong Restaurant,Vietnamese Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Sushi Restaurant,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Beer Store,Fried Chicken Joint,Sandwich Place,Japanese Restaurant,Coffee Shop,Fast Food Restaurant,Video Store,Pharmacy
4,"Alderwood, Long Branch",Pub,Gym,Dance Studio,Sandwich Place,Women's Store,Discount Store,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [31]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 4, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [33]:
print(neighborhoods_venues_sorted.columns.tolist())
neighborhoods_venues_sorted.head()

['Cluster Labels', 'Neighborhood', '1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', '9th Most Common Venue', '10th Most Common Venue']


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide, King, Richmond",Café,Coffee Shop,Hotel,Burger Joint,Bar,Bakery,Asian Restaurant,Steakhouse,Japanese Restaurant,Restaurant
1,0,Agincourt,Supermarket,Shopping Mall,Park,Pool,Sushi Restaurant,Chinese Restaurant,Shanghai Restaurant,Bakery,Hong Kong Restaurant,Vietnamese Restaurant
2,0,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Sushi Restaurant,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,0,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Beer Store,Fried Chicken Joint,Sandwich Place,Japanese Restaurant,Coffee Shop,Fast Food Restaurant,Video Store,Pharmacy
4,0,"Alderwood, Long Branch",Pub,Gym,Dance Studio,Sandwich Place,Women's Store,Discount Store,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [34]:
toronto_merged = df3
# Rename column to match joining dataframes
toronto_merged = toronto_merged.rename(columns={'Neighbourhood':'Neighborhood'})

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Lattitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750,0.0,History Museum,Bar,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256,4.0,Construction & Landscaping,Gym / Fitness Center,Park,Dog Run,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,M1G,Scarborough,Woburn,43.768359,-79.217590,0.0,Korean Restaurant,Park,Coffee Shop,Business Service,Women's Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440,0.0,Playground,Trail,Dog Run,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [35]:
## Drop nulls

toronto_merged = toronto_merged.dropna(subset=['Cluster Labels'])

## Convert Cluster Labels to int
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

toronto_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 1 to 102
Data columns (total 16 columns):
Postcode                  100 non-null object
Borough                   100 non-null object
Neighborhood              100 non-null object
Lattitude                 100 non-null float64
Longitude                 100 non-null float64
Cluster Labels            100 non-null int64
1st Most Common Venue     100 non-null object
2nd Most Common Venue     100 non-null object
3rd Most Common Venue     100 non-null object
4th Most Common Venue     100 non-null object
5th Most Common Venue     100 non-null object
6th Most Common Venue     100 non-null object
7th Most Common Venue     100 non-null object
8th Most Common Venue     100 non-null object
9th Most Common Venue     100 non-null object
10th Most Common Venue    100 non-null object
dtypes: float64(2), int64(1), object(13)
memory usage: 13.3+ KB


Finally, let's visualize the resulting clusters

In [36]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Lattitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0,History Museum,Bar,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,Scarborough,0,Korean Restaurant,Park,Coffee Shop,Business Service,Women's Store,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
4,Scarborough,0,Playground,Trail,Dog Run,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
5,Scarborough,0,Indian Restaurant,Train Station,Grocery Store,Restaurant,Women's Store,Dog Run,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
6,Scarborough,0,Discount Store,Hobby Shop,Coffee Shop,Department Store,Chinese Restaurant,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
7,Scarborough,0,Bakery,Bus Line,Coffee Shop,Soccer Field,Metro Station,Bus Station,Intersection,Fish Market,Fish & Chips Shop,Field
8,Scarborough,0,Fast Food Restaurant,Sandwich Place,Pizza Place,Pharmacy,Coffee Shop,Liquor Store,Furniture / Home Store,Discount Store,Eastern European Restaurant,Field
9,Scarborough,0,College Stadium,Gym Pool,Gym,Skating Rink,Park,General Entertainment,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market
10,Scarborough,0,Brewery,Gift Shop,Light Rail Station,Bakery,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
12,Scarborough,0,Supermarket,Shopping Mall,Park,Pool,Sushi Restaurant,Chinese Restaurant,Shanghai Restaurant,Bakery,Hong Kong Restaurant,Vietnamese Restaurant


#### Cluster 2

In [103]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,North York,1,Construction & Landscaping,Burger Joint,Park,Intersection,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
34,North York,1,Food Stand,French Restaurant,Intersection,Park,Grocery Store,Women's Store,Dog Run,Field,Fast Food Restaurant,Farmers Market


#### Cluster 3

In [104]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,2,Convenience Store,Auto Garage,Women's Store,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
23,North York,2,Speakeasy,Park,Bank,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
40,East York,2,Convenience Store,Park,Italian Restaurant,Farmers Market,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
44,Central Toronto,2,Park,Convenience Store,Grocery Store,Bank,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
45,Central Toronto,2,Park,Convenience Store,Grocery Store,Bank,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
46,Central Toronto,2,Park,Convenience Store,Grocery Store,Bank,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
47,Central Toronto,2,Park,Convenience Store,Grocery Store,Bank,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
48,Central Toronto,2,Park,Convenience Store,Grocery Store,Bank,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
49,Central Toronto,2,Park,Convenience Store,Grocery Store,Bank,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
63,Central Toronto,2,Park,Convenience Store,Grocery Store,Bank,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market


#### Cluster 4

In [105]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,North York,3,Music Venue,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


#### Cluster 5

In [106]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,4,Construction & Landscaping,Gym / Fitness Center,Park,Dog Run,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
19,North York,4,Construction & Landscaping,Dog Run,Park,Trail,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
31,North York,4,Gym / Fitness Center,Mobile Phone Shop,Park,Women's Store,Dog Run,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
32,North York,4,Construction & Landscaping,Park,Business Service,Moving Target,Electronics Store,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
96,North York,4,Construction & Landscaping,Rental Car Location,Dog Run,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


#### References

[1] Data & The World: https://www.dataandtheworld.com/2017/06/20/scraping-wikipedia-tables-python-r/